# The fractional knapsack problem

Sia dato uno zaino che possa sopportare un determinato peso e siano dati $n$ oggetti, ognuno dei quali caratterizzato da un peso e un valore. Il problema si propone di scegliere quali di questi oggetti mettere nello zaino per ottenere il maggiore valore senza eccedere il peso sostenibile dallo zaino stesso.

<img src="https://tommasoadamo.it/images/lez17/knapsack.png" style="width: 400px" />

Supponiamo che gli oggetti possano essere frazionati. 

### LP Formulation

\begin{align}
\max \quad& \sum\limits_{i \in N} c_i x_i\\
\quad& \sum\limits_{i \in N} w_i x_i \leq W\\
\quad& 0 \leq x_i \leq 1 & i \in N
\end{align}

### Problem Instance: oggetti.json

Optimal solution with total cost: 7566.857

In [ ]:
# ALERT: execute this cell to prepare input data!
import requests
def download(link, nomeFile=None):
    if nomeFile == None:
        nomeFile = link.split('/')[-1]
    richiesta = requests.get(link)
    if richiesta.status_code == 200:
        with open(nomeFile, 'w') as file:
            file.write(richiesta.text)
            
download('https://tommasoadamo.it/data/oggetti.json')

In [ ]:
# ALERT: execute this cell to install DOcplex! 
!pip install docplex cplex

In [ ]:
import docplex.mp.model as cplex
import json

# prepariamo l'istanza
with open('oggetti.json', 'r') as f:
    D = json.load(f)

W = D['zaino']['capacita']

c = []
w = []    
for o in D['oggetti']:
    c.append(o['costo'])
    w.append(o['peso'])
n = len(c)
R = tuple(range(n))

# codifichiamo il modello
with cplex.Model("Fractional knapsack") as mdl:
    x = mdl.continuous_var_list(R, name="x")

    mdl.add_constraint(sum(w[i] * x[i] for i in R) <= W)
    
    for i in R:
        mdl.add_constraint(x[i] <= 1)
    
    mdl.maximize(sum(c[i] * x[i] for i in R))
        
    mdl.print_information()
    sol = mdl.solve(log_output=True)
    
    if sol == None:
        print('No solution.')
    else:
        print(sol)
        
        # presentiamo la soluzione
        packed_items = [i for i in R if x[i].solution_value > 0]
        print('Packed items: ' + str(packed_items))
        tw = sum(w[i] * x[i].solution_value for i in packed_items)
        print('Total weight:', tw)
        print('Total cost:', sol.objective_value)